<a href="https://colab.research.google.com/github/RajuDasa/llm_engineering/blob/day2_branch/week1/community-contributions/week1_assignments_raju/llm_week1_day2_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment:** summarize a webpage using Ollama and openAI library.

In [ ]:
#install ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
# Start Ollama server

import subprocess

# Start the command in a background process
process = subprocess.Popen(["ollama", "serve"])

# The kernel can continue execution while the process runs in the background
print("The 'ollama serve' process is running in the background.")

In [ ]:
#download llama model
!ollama pull llama3.2

In [ ]:
#test server
import requests

requests.get("http://localhost:11434").content

In [ ]:
!pip install OpenAI
!pip install beautifulsoup4
#already installed

In [ ]:
#Test with library

# from openai import OpenAI

# OLLAMA_BASE_URL = "http://localhost:11434/v1"

# ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
# response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])
# display(response.choices[0].message.content)

In [ ]:
#scraping logic:

from bs4 import BeautifulSoup
import requests


# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]


In [ ]:
#Now get summary of a website using our model

system_prompt = """
You are a helpful assistant that analyzes the contents of a website,
and provides a short, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

def summarize(url):
    website = fetch_website_contents(url)
    response = ollama.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content


In [ ]:
from IPython.display import Markdown, display

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://edwarddonner.com")


Took 2mins to execute and returned only 330 char summary without humor.

In [ ]:
fetch_website_contents("https://edwarddonner.com") #2000 char

In [ ]:
#stop ollama server
process.kill()